In [319]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import nltk 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
import pandas

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def readGz(path):
  for l in gzip.open(path, 'rt'):
    yield eval(l)

def readCSV(path):
  f = gzip.open(path, 'rt')
  f.readline()
  for l in f:
    yield l.strip().split(',')

In [4]:
def accuracy(predictions, y):
    correct = predictions == y
    return sum(correct) / len(correct)

In [25]:
allRatings = []
userRatings = defaultdict(list)

for user,book,r in readCSV("train_Interactions.csv.gz"):
  r = int(r)
  allRatings.append(r)
  userRatings[user].append(r)

In [26]:
globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
  userAverage[u] = sum(userRatings[u]) / len(userRatings[u])

In [27]:
readBookAvg = []
for u in userRatings:
    readBookAvg.append(len(userRatings[u]))
readBookAvg = sum(readBookAvg)/len(readBookAvg)

In [28]:
wholeRatings = []
for d in readCSV("train_Interactions.csv.gz"):
    d[2] = int(d[2])
    wholeRatings.append(d)

In [29]:
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in wholeRatings:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [30]:
# === Read Prediction === #

In [31]:
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
  bookCount[book] += 1
  totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()


In [54]:
return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > 1.5*totalRead/2: break

In [33]:
# ==== Testing Starts ==== #

In [34]:
#for testing
ratingsTrain = wholeRatings[:190000]
ratingsTest = wholeRatings[190000:]
itemsPerUser = defaultdict(set)
usersPerItem = defaultdict(set)
for u,b,r in ratingsTrain:
    itemsPerUser[u].add(b)
    usersPerItem[b].add(u)

In [37]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [38]:
def JaccardSim(u,b):
    maxSim = 0
    for b2 in itemsPerUser[u]:
        if b2==b:
            continue
        temp = Jaccard(usersPerItem[b],usersPerItem[b2])
        if temp>maxSim:
            maxSim = temp
    return maxSim

In [39]:
rI=defaultdict(set)
rU=defaultdict(set)
for u,b,r in wholeRatings:
    rI[b].add(u)
    rU[u].add(b)

In [44]:
notRead = defaultdict(list)
items = set(rI.keys())
for u in rU:
    notRead[u]=items.difference(rU[u])

In [45]:
def read_pred(u,b):
    a = b in return1
    return [1,a,JaccardSim(u,b),len(itemsPerUser[u]),len(usersPerItem[b])]

In [46]:
trainNeg=[]
users=list(rU.keys())
for u,i,_ in ratingsTrain:
    samp = notRead[u]
    r = random.choice(list(samp))
    trainNeg.append((u,r,0))

In [55]:
train = ratingsTrain + trainNeg
train = [read_pred(u,b) for u,b,r in train]
yTrain =[1]*len(ratingsTrain) + [0]*len(trainNeg)

In [57]:
validNeg=[]
users=list(rU.keys())
for u,i,_ in ratingsTest:
    samp = notRead[u]
    r = random.choice(list(samp))
    validNeg.append((u,r,0))

In [58]:
val = ratingsTest + validNeg
y=[1]*10000 + [0]*10000
val[0]

['u59070515', 'b55084829', 5]

In [59]:
valid = [read_pred(u,b) for u,b,r in val]
valid[:10]

[[1, False, 0.0017301038062283738, 15, 14],
 [1, True, 0.011655011655011656, 15, 299],
 [1, False, 0.004629629629629629, 8, 22],
 [1, True, 0.01990049751243781, 8, 114],
 [1, True, 0.006060606060606061, 9, 392],
 [1, True, 0.004070556309362279, 6, 53],
 [1, True, 0.00851063829787234, 16, 174],
 [1, False, 0.003105590062111801, 17, 16],
 [1, True, 0.02631578947368421, 21, 66],
 [1, True, 0.018361581920903956, 9, 580]]

In [61]:
accList=[]
for c in [-9,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5]:
    mod = linear_model.LogisticRegression(C=10**c)
    mod.fit(train,yTrain)
    tempPred = mod.predict(valid)
    acc = accuracy(tempPred,y)
    accList.append(acc)
accList

[0.7058,
 0.72035,
 0.72665,
 0.7271,
 0.731,
 0.75015,
 0.7532,
 0.75315,
 0.749,
 0.72665,
 0.71565,
 0.71255,
 0.71225,
 0.7122,
 0.7122]

In [67]:
mod = linear_model.LogisticRegression(C=10**-3)
mod.fit(train,yTrain)

LogisticRegression(C=0.001)

In [68]:
read_prediction = mod.predict(valid)

In [69]:
base_read_prediction = [b in return1 for u,b,_ in val]

In [70]:
accForBase = accuracy(read_prediction,y)
accForBase

0.7532

In [ ]:
# ==== Testing Ends ==== #

In [334]:
# ==== Actual Starts ====#

In [71]:
#for hidden
itemsPerUser = defaultdict(set)
usersPerItem = defaultdict(set)
for u,b,r in wholeRatings:
    itemsPerUser[u].add(b)
    usersPerItem[b].add(u)

In [72]:
def read_pred(u,b):
    a = b in return1
    return [1,a,JaccardSim(u,b),len(itemsPerUser[u]),len(usersPerItem[b])]

In [74]:
realNeg=[]
users=list(rU.keys())
for u,i,_ in wholeRatings:
    samp = notRead[u]
    r = random.choice(list(samp))
    realNeg.append((u,r,0))

In [75]:
realTrain = wholeRatings + realNeg
realTrain = [read_pred(u,b) for u,b,r in realTrain]
realY =[1]*len(wholeRatings) + [0]*len(realNeg)

In [76]:
realTrain[:10]

[[1, True, 0.018066847335140017, 18, 463],
 [1, False, 0.058823529411764705, 7, 10],
 [1, False, 0.037037037037037035, 20, 14],
 [1, True, 0.017094017094017096, 10, 93],
 [1, True, 0.019230769230769232, 9, 111],
 [1, True, 0.01948051948051948, 5, 79],
 [1, True, 0.0031446540880503146, 4, 311],
 [1, True, 0.02356902356902357, 14, 137],
 [1, False, 0.024390243902439025, 6, 15],
 [1, True, 0.022388059701492536, 10, 87]]

In [77]:
mod = linear_model.LogisticRegression(C=10**-3)
mod.fit(realTrain,realY)

LogisticRegression(C=0.001)

In [78]:
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,b = l.strip().split(',')
  if mod.predict([read_pred(u,b)])[0]:
    predictions.write(u + ',' + b + ",1\n")
  else:
    predictions.write(u + ',' + b + ",0\n")

predictions.close()

In [79]:
# === Category Prediction === #

In [5]:
catDict = {
  "children": 0,
  "comics_graphic": 1,
  "fantasy_paranormal": 2,
  "mystery_thriller_crime": 3,
  "young_adult": 4
}

In [6]:
data = []

for d in readGz("train_Category.json.gz"):
    data.append(d)

In [7]:
def cleaning(txt):
    newTxt = txt.translate(str.maketrans('', '', string.punctuation))
    return newTxt.lower()

In [8]:
dataTrain = data[:90000]
dataTest = data[90000:]

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sunwoo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
sw = stopwords.words("english")

In [265]:
data[0]

{'user_id': 'u75242413',
 'review_id': 'r45843137',
 'rating': 4,
 'review_text': "a clever book with a deeply troubling premise and an intriguing protagonist. Thompson's clean, sparse prose style kept each page feeling light even as some rather heavy existential questions dropped upon them. I enjoyed it. \n and that cover design is boom-pow gorgeous.",
 'n_votes': 1,
 'genre': 'mystery_thriller_crime',
 'genreID': 3}

In [11]:
wordCount=defaultdict(int)
for d in dataTrain:
    txt = cleaning(d['review_text'])
    for i in txt.split():
        if i in sw:
            continue
        wordCount[i]+=1

In [12]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [85]:
words = [d[1] for d in counts[:1500]]

In [88]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [249]:
grams = [x[1] for x in gcounts[:2000]]

In [250]:
gramId = dict(zip(grams, range(len(grams))))

In [ ]:
#tfidf

In [332]:
tftrain = [d['review_text'] for d in dataTrain]
tftest = [d['review_text'] for d in dataTest]
cv=CountVectorizer(stop_words='english')
cvVec = cv.fit_transform(tftrain)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(cvVec)

TfidfTransformer()

In [375]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True,stop_words='english') 
fitted_vectorizer=tfidf_vectorizer.fit(tftrain)
tfidfTrain=fitted_vectorizer.transform(tftrain)

In [382]:
clf = linear_model.LogisticRegression(C=1000)
clf.fit(tfidfTrain, ytrain)

LogisticRegression(C=1000)

In [377]:
tfidfTest=fitted_vectorizer.transform(tftest)

In [383]:
tfpred = clf.predict(tfidfTest)
accuracy(tfpred,yvalid)

0.7675

In [385]:
tfpred

array([3, 3, 2, ..., 3, 3, 2])

In [380]:
accList = []
for c in [0.0001, 0.001, 0.01, 0.1, 1, 10 ,100, 1000, 10000]:
    clf = linear_model.LogisticRegression(C=c)
    clf.fit(tfidfTrain, ytrain)
    pred = clf.predict(tfidfTest)
    correct = pred == yvalid
    acc1 = sum(correct) / len(correct)
    accList.append(acc1)

In [381]:
accList

[0.3593, 0.3594, 0.525, 0.7094, 0.7616, 0.7641, 0.7618, 0.7675, 0.7615]

In [275]:
def cat_pred(datum):
    feat = [0]*len(grams)
    r = cleaning(datum['review_text'])
    ws = r.split()
    ws = [d for d in ws if d not in sw ]
    ws2 = [' '.join(x) for x in list(zip(ws[:-1],ws[1:]))]
    for w in ws + ws2:
        if w in grams:
            feat[gramId[w]] += 1
    gs = [0,0,0,0,0]
    for i in userGenres[datum['user_id']]:
        gs[i] += 1
    feat += gs
    feat.append(datum['n_votes'])
    feat.append(1)
    return feat

In [201]:
userGenres=defaultdict(list)
for d in dataTrain:
    userGenres[d['user_id']].append(d['genreID'])

In [276]:
X = [cat_pred(d) for d in data]

In [277]:
y=[d['genreID'] for d in data]

In [278]:
Xtrain = X[:9*len(X)//10]
ytrain = y[:9*len(y)//10]
Xvalid = X[9*len(X)//10:]
yvalid = y[9*len(y)//10:]

In [255]:
accList = []
for c in [0.0001, 0.001, 0.01, 0.1, 1, 10 ,100, 1000, 10000]:
    clf = linear_model.LogisticRegression(C=c)
    clf.fit(Xtrain, ytrain)
    pred = clf.predict(Xvalid)
    correct = pred == yvalid
    acc1 = sum(correct) / len(correct)
    accList.append(acc1)

In [256]:
accList

[0.549, 0.6225, 0.681, 0.6962, 0.6933, 0.6921, 0.6924, 0.6922, 0.6924]

In [279]:
clf = linear_model.LogisticRegression(C=0.1)
clf.fit(Xtrain, ytrain)

LogisticRegression(C=0.1)

In [280]:
pred = clf.predict(Xvalid)
accuracy(pred,yvalid)

0.6979

In [124]:
# ==== Category actual ==== #

In [282]:
grams = [x[1] for x in gcounts[:2000]]

In [283]:
gramId = dict(zip(grams, range(len(grams))))

In [284]:
def cat_pred(datum):
    feat = [0]*len(grams)
    r = cleaning(datum['review_text'])
    ws = r.split()
    ws = [d for d in ws if d not in sw ]
    ws2 = [' '.join(x) for x in list(zip(ws[:-1],ws[1:]))]
    for w in ws + ws2:
        if w in grams:
            feat[gramId[w]] += 1
    gs = [0,0,0,0,0]
    for i in userGenres[datum['user_id']]:
        gs[i] += 1
    feat += gs
    feat.append(datum['n_votes'])
    feat.append(1)
    return feat

In [285]:
X = [cat_pred(d) for d in data]
y = [d['genreID'] for d in data]

In [286]:
clf = linear_model.LogisticRegression(C=0.1)
clf.fit(X, y)

LogisticRegression(C=0.1)

In [ ]:
# ==== tfidf ==== #

In [390]:
tffull = [d['review_text'] for d in data]
y = [d['genreID'] for d in data]

In [389]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True,stop_words='english') 
fitted_vectorizer=tfidf_vectorizer.fit(tffull)
tfidfFull=fitted_vectorizer.transform(tffull)

In [391]:
clf = linear_model.LogisticRegression(C=1000)
clf.fit(tfidfFull, y)

LogisticRegression(C=1000)

In [ ]:
tfidfTest=fitted_vectorizer.transform(res)

In [400]:
predictions = open("predictions_Category.csv", 'w')
predictions.write("userID,reviewID,prediction\n")
res = []
output = []
for l in readGz("test_Category.json.gz"):
    res.append(l['review_text'])
    output.append((l['user_id'],l['review_id']))
    
res = tfidfTest=fitted_vectorizer.transform(res)
pred = clf.predict(res)

for o in zip(output,pred):
    predictions.write(o[0][0] + ',' + o[0][1] + "," + str(o[1]) + "\n")

In [401]:
predictions.close()